In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf

from tqdm import tqdm
from keras import optimizers
from keras.models import Sequential
from keras.layers import AveragePooling2D, Flatten, Dropout, Dense
from keras.applications import InceptionV3
from keras.applications import imagenet_utils
from keras.applications.inception_v3 import preprocess_input

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator

from constants import LABEL_DIR, RAW_IMAGES_DIR, FEATURES_DIR, TOTAL_N, SUBMISSION_FILE_DIR

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#stolen from https://stackoverflow.com/questions/35365007/tensorflow-precision-recall-f1-score-and-confusion-matrix
def tf_f1_score(y_true, y_pred):
    """Computes 3 different f1 scores, micro macro
    weighted.
    micro: f1 score accross the classes, as 1
    macro: mean of f1 scores per class
    weighted: weighted average of f1 scores per class,
            weighted from the support of each class


    Args:
        y_true (Tensor): labels, with shape (batch, num_classes)
        y_pred (Tensor): model's predictions, same shape as y_true

    Returns:
        tuple(Tensor): (micro, macro, weighted)
                    tuple of the computed f1 scores
    """

    f1s = [0, 0, 0]

    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)

    for i, axis in enumerate([None, 0]):
        TP = tf.count_nonzero(y_pred * y_true, axis=axis)
        FP = tf.count_nonzero(y_pred * (y_true - 1), axis=axis)
        FN = tf.count_nonzero((y_pred - 1) * y_true, axis=axis)

        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        f1 = 2 * precision * recall / (precision + recall)

        f1s[i] = tf.reduce_mean(f1)

    weights = tf.reduce_sum(y_true, axis=0)
    weights /= tf.reduce_sum(weights)

    f1s[2] = tf.reduce_sum(f1 * weights)

    micro, macro, weighted = f1s
    return micro

In [3]:
dummy_cnn_features = np.random.rand(1000,2048)
annotations = np.load('../data/labels/annotations.npz')['arr_0'][:1000]

In [22]:
top_model = Sequential()
top_model.add(Dropout(0.5, input_shape=(2048,)))
top_model.add(Dense(229, activation='sigmoid'))

top_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy',tf_f1_score])

In [28]:
top_model.fit(dummy_cnn_features,annotations,batch_size=100,nb_epoch=30)


C:\ProgramData\Anaconda3\lib\site-packages\keras\models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/30
1000/1000 [==============================] - ETA: 0s - loss: 0.1040 - acc: 0.9748 - tf_f1_score: 0.050 - ETA: 0s - loss: 0.1008 - acc: 0.9758 - tf_f1_score: 0.048 - ETA: 0s - loss: 0.1022 - acc: 0.9754 - tf_f1_score: 0.048 - ETA: 0s - loss: 0.1015 - acc: 0.9754 - tf_f1_score: 0.048 - ETA: 0s - loss: 0.1002 - acc: 0.9756 - tf_f1_score: 0.048 - 0s 321us/step - loss: 0.0994 - acc: 0.9757 - tf_f1_score: 0.0479
Epoch 2/30
1000/1000 [==============================] - ETA: 0s - loss: 0.0917 - acc: 0.9769 - tf_f1_score: 0.045 - ETA: 0s - loss: 0.0940 - acc: 0.9764 - tf_f1_score: 0.046 - ETA: 0s - loss: 0.0952 - acc: 0.9760 - tf_f1_score: 0.047 - ETA: 0s - loss: 0.0954 - acc: 0.9760 - tf_f1_score: 0.047 - ETA: 0s - loss: 0.0946 - acc: 0.9759 - tf_f1_score: 0.047 - 0s 323us/step - loss: 0.0948 - acc: 0.9759 - tf_f1_score: 0.0479
Epoch 3/30
1000/1000 [==============================] - ETA: 0s - loss: 0.0929 - acc: 0.9756 - tf_f1_score: 0.048 - ETA: 0s - loss: 0.0927 - acc: 0.9759 - tf_

1000/1000 [==============================] - ETA: 0s - loss: 0.0711 - acc: 0.9780 - tf_f1_score: 0.047 - ETA: 0s - loss: 0.0733 - acc: 0.9772 - tf_f1_score: 0.047 - ETA: 0s - loss: 0.0731 - acc: 0.9773 - tf_f1_score: 0.047 - ETA: 0s - loss: 0.0727 - acc: 0.9772 - tf_f1_score: 0.047 - ETA: 0s - loss: 0.0725 - acc: 0.9774 - tf_f1_score: 0.047 - 0s 339us/step - loss: 0.0727 - acc: 0.9774 - tf_f1_score: 0.0479
Epoch 21/30
1000/1000 [==============================] - ETA: 0s - loss: 0.0682 - acc: 0.9783 - tf_f1_score: 0.047 - ETA: 0s - loss: 0.0704 - acc: 0.9781 - tf_f1_score: 0.047 - ETA: 0s - loss: 0.0706 - acc: 0.9776 - tf_f1_score: 0.047 - ETA: 0s - loss: 0.0711 - acc: 0.9774 - tf_f1_score: 0.048 - ETA: 0s - loss: 0.0714 - acc: 0.9773 - tf_f1_score: 0.048 - 0s 351us/step - loss: 0.0715 - acc: 0.9774 - tf_f1_score: 0.0479
Epoch 22/30
1000/1000 [==============================] - ETA: 0s - loss: 0.0712 - acc: 0.9775 - tf_f1_score: 0.047 - ETA: 0s - loss: 0.0706 - acc: 0.9779 - tf_f1_score:

In [24]:
score = top_model.evaluate(dummy_cnn_features, annotations, verbose=0)
score

[0.09832346898317337, 0.97689954662323, 0.04785915269761267]

In [25]:
predictions = top_model.predict(dummy_cnn_features)

In [26]:


def to_label_list(n_hot):
  """
  transforms a n-hot-matrix into a list of labels
  :return:
  """
  return np.array([np.where(labels == 1)[0] for labels in n_hot])

def make_kaggle_sub_file(predictions):
  #assert predictions.shape[0] == TOTAL_N['test']
  
  predictions = predictions.round(0).astype(np.uint8)
  
  test_image_ids = range(1,TOTAL_N['test']+1)

  print('Creating submission file in %s..'%SUBMISSION_FILE_DIR)
  if not os.path.exists(SUBMISSION_FILE_DIR):
    os.makedirs(SUBMISSION_FILE_DIR)
  labels = to_label_list(predictions[1:])
  out = np.c_[test_image_ids,labels]
  out = [','.join([str(row[0]), ' '.join(list(map(str,row[1])))]) for row in out]
  file_name = pd.Timestamp.now().strftime('%Y%m%d_%H%M') + '_submission.csv'
  with open(os.path.join(SUBMISSION_FILE_DIR,file_name), 'w+') as f:
    f.write('image_id,label_id\n')
    for row in out:
      f.write(row+'\n')
  print('File Saved')
make_kaggle_sub_file(predictions)

Creating submission file in ../data/submissions..
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
File Saved
